In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Construct the relative path and load the CSV
df = pd.read_csv(os.path.join(current_directory, "../1_Preprocessing Notebooks/Final Sheets/transactions_details.csv"))
df.head()

,transaction_id,unique_product_id,customer_id,transaction_date,online_order,order_status,list_price,standard_cost,profit
0,1,P00211222012,2950,2017-02-25,0,Approved,71.49,53.62,17.87
1,2,P00321232014,3120,2017-05-21,1,Approved,2091.47,388.92,1702.55
2,3,P03731121999,402,2017-10-16,0,Approved,1793.43,248.82,1544.61
3,4,P08841221998,3135,2017-08-31,0,Approved,1198.46,381.10,817.36
4,5,P07851232015,787,2017-10-01,1,Approved,1765.30,709.48,1055.82


In [3]:
df['order_status'].value_counts()

order_status
Approved     19821
Cancelled      179
Name: count, dtype: int64

In [4]:
df.drop(df[df['order_status']=='Cancelled'].index, inplace=True)
df['order_status'].value_counts()

order_status
Approved    19821
Name: count, dtype: int64

In [5]:
df.isnull().sum()

transaction_id       0
unique_product_id    0
customer_id          0
transaction_date     0
online_order         0
order_status         0
list_price           0
standard_cost        0
profit               0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19821 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     19821 non-null  int64  
 1   unique_product_id  19821 non-null  object 
 2   customer_id        19821 non-null  int64  
 3   transaction_date   19821 non-null  object 
 4   online_order       19821 non-null  int64  
 5   order_status       19821 non-null  object 
 6   list_price         19821 non-null  float64
 7   standard_cost      19821 non-null  float64
 8   profit             19821 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 1.5+ MB


In [7]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19821 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   transaction_id     19821 non-null  int64         
 1   unique_product_id  19821 non-null  object        
 2   customer_id        19821 non-null  int64         
 3   transaction_date   19821 non-null  datetime64[ns]
 4   online_order       19821 non-null  int64         
 5   order_status       19821 non-null  object        
 6   list_price         19821 non-null  float64       
 7   standard_cost      19821 non-null  float64       
 8   profit             19821 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 1.5+ MB


In [9]:
df['transaction_date'].max()

Timestamp('2017-12-30 00:00:00')

## **RFM Calculation**

#### 1. **Recency:** Calculate the difference in days between the most recent transaction and current date for each customer

In [10]:
current_date = pd.to_datetime("2017-12-31")

recency_df = df.groupby('customer_id').agg({'transaction_date': lambda x: (current_date - x.max()).days}).reset_index()
recency_df.columns = ['customer_id', 'recency']
recency_df.head()

,customer_id,recency
0,1,8
1,2,129
2,3,103
3,4,196
4,5,17


#### 2. **Frequency:** Count the number of transactions for each customer

In [11]:
frequency_df = df.groupby('customer_id').agg({'transaction_id': 'count'}).reset_index()
frequency_df.columns = ['customer_id', 'frequency']
frequency_df.head()

,customer_id,frequency
0,1,11
1,2,3
2,3,8
3,4,2
4,5,6


#### 3. **Monetary:** Sum the total spending (list_price) for each customer

In [12]:
monetary_df = df.groupby('customer_id').agg({
    'list_price': 'sum'
}).reset_index()
monetary_df.columns = ['customer_id', 'monetary']
monetary_df.head()

,customer_id,monetary
0,1,9084.45
1,2,4149.07
2,3,9888.23
3,4,1047.72
4,5,5903.20


### Merge the R, F, M dataframes into one

In [13]:
rfm_df = recency_df.merge(frequency_df, on='customer_id').merge(monetary_df, on='customer_id')
rfm_df.head()

,customer_id,recency,frequency,monetary
0,1,8,11,9084.45
1,2,129,3,4149.07
2,3,103,8,9888.23
3,4,196,2,1047.72
4,5,17,6,5903.20


### Normalize the RFM scores by ranking customers into quintiles (or assign scores directly)

In [14]:
rfm_df['R_rank'] = pd.qcut(rfm_df['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm_df['F_rank'] = pd.qcut(rfm_df['frequency'], 5, labels=[1, 2, 3, 4, 5])
rfm_df['M_rank'] = pd.qcut(rfm_df['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm_df.head()

,customer_id,recency,frequency,monetary,R_rank,F_rank,M_rank
0,1,8,11,9084.45,5,5,5
1,2,129,3,4149.07,1,1,2
2,3,103,8,9888.23,1,4,5
3,4,196,2,1047.72,1,1,1
4,5,17,6,5903.20,4,3,3


## Combine the RFM scores

In [15]:
rfm_df['RFM_rank'] = rfm_df['R_rank'].astype(str) + rfm_df['F_rank'].astype(str) + rfm_df['M_rank'].astype(str)

rfm_df.head()

,customer_id,recency,frequency,monetary,R_rank,F_rank,M_rank,RFM_rank
0,1,8,11,9084.45,5,5,5,555
1,2,129,3,4149.07,1,1,2,112
2,3,103,8,9888.23,1,4,5,145
3,4,196,2,1047.72,1,1,1,111
4,5,17,6,5903.20,4,3,3,433


In [16]:
seg_map = {
    r'[4-5][4-5]': 'Champions',            # High recency and frequency
    r'[3-4][3-5]': 'Loyal Customers',      # Moderately high recency and frequency
    r'[4-5][2-3]': 'Potential Loyalists',  # High recency but moderate frequency
    r'[1-2][3-5]': 'At Risk',              # Low recency but moderate to high frequency
    r'[1-2][1-2]': 'Hibernating',          # Low recency and frequency
    r'3[1-2]': 'New Customers',            # Moderate recency but low frequency (example)
    r'[4-5]1': 'Promising'                 # High recency but very low frequency (example)
}

rfm_df['Segment'] = rfm_df['R_rank'].astype(str) + rfm_df['F_rank'].astype(str)
rfm_df['Segment'] = rfm_df['Segment'].replace(seg_map, regex=True)

rfm_df_segment = rfm_df.drop(['R_rank', 'F_rank', 'M_rank'], axis=1)

rfm_df_segment.head()

,customer_id,recency,frequency,monetary,RFM_rank,Segment
0,1,8,11,9084.45,555,Champions
1,2,129,3,4149.07,112,Hibernating
2,3,103,8,9888.23,145,At Risk
3,4,196,2,1047.72,111,Hibernating
4,5,17,6,5903.20,433,Loyal Customers


In [17]:
rfm_df['Segment'].value_counts()

Segment
Hibernating            897
Champions              618
Loyal Customers        508
At Risk                494
Potential Loyalists    362
Promising              320
New Customers          294
Name: count, dtype: int64

In [18]:
import plotly.express as px

segment_counts = rfm_df['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']

fig = px.pie(segment_counts, values='Count', names='Segment', 
             title='Customer Segments Distribution',
             color_discrete_sequence=px.colors.qualitative.Vivid_r)
fig.show()

In [19]:
import plotly.express as px

fig = px.bar(segment_counts, x='Segment', y='Count', 
             title='Number of Customers by Segment',
             color='Segment', 
             color_discrete_sequence=px.colors.qualitative.Vivid_r)
fig.update_layout(xaxis_title="Customer Segment", yaxis_title="Number of Customers")
fig.show()

In [20]:
import plotly.express as px

# Box plot of monetary value for each segment
fig = px.box(rfm_df, x='Segment', y='monetary', 
             title='Monetary Value Distribution per Segment',
             color='Segment', 
             color_discrete_sequence=px.colors.qualitative.Vivid_r)
fig.update_layout(xaxis_title="Customer Segment", yaxis_title="Monetary Value")
fig.show()

In [21]:
import plotly.express as px

# Treemap of customer segments
fig = px.treemap(segment_counts, path=['Segment'], values='Count', 
                 title='Customer Segmentation Overview', 
                 color='Count',
                 color_discrete_sequence=px.colors.qualitative.Vivid_r)
fig.show()

# **LTV Analysis**

In [22]:
# Calculate total spending per customer and total transactions per customer
total_spent_per_customer = df.groupby('customer_id')['list_price'].sum()
total_transactions_per_customer = df.groupby('customer_id')['transaction_id'].count()

In [23]:
# Calculate AOV per customer
aov_per_customer = total_spent_per_customer / total_transactions_per_customer

In [24]:
first_purchase_date = df.groupby('customer_id')['transaction_date'].min()
last_purchase_date = df.groupby('customer_id')['transaction_date'].max()

In [25]:
# Calculate customer lifespan (in years)
customer_lifespan = (last_purchase_date - first_purchase_date).dt.days / 365

In [26]:
# Calculate LTV per customer
ltv_per_customer = aov_per_customer * total_transactions_per_customer * customer_lifespan

ltv_df = pd.DataFrame({
    'customer_id': ltv_per_customer.index,
    'LTV': ltv_per_customer
}).reset_index(drop=True)
ltv_df

,customer_id,LTV
0,1,8760.894247
1,2,1273.139288
2,3,5634.936548
3,4,218.155397
4,5,4625.521096
...,...,...
3488,3497,697.525370
3489,3498,2879.296384
3490,3499,6327.993096
3491,3500,2845.557562


In [27]:
# Merge RFM and LTV
rfm_ltv_df = rfm_df_segment.merge(ltv_df, on='customer_id')
rfm_ltv_df

,customer_id,recency,frequency,monetary,RFM_rank,Segment,LTV
0,1,8,11,9084.45,555,Champions,8760.894247
1,2,129,3,4149.07,112,Hibernating,1273.139288
2,3,103,8,9888.23,145,At Risk,5634.936548
3,4,196,2,1047.72,111,Hibernating,218.155397
4,5,17,6,5903.20,433,Loyal Customers,4625.521096
...,...,...,...,...,...,...,...
3488,3497,53,3,3744.07,311,New Customers,697.525370
3489,3498,128,6,5177.06,132,At Risk,2879.296384
3490,3499,52,7,7673.48,344,Loyal Customers,6327.993096
3491,3500,145,6,4922.41,132,At Risk,2845.557562


In [28]:
rfm_ltv_df.columns

Index(['customer_id', 'recency', 'frequency', 'monetary', 'RFM_rank',
       'Segment', 'LTV'],
      dtype='object')

In [29]:
# Scatter plot showing LTV and RFM segments
fig = px.scatter(rfm_ltv_df, 
                 x='recency', 
                 y='LTV', 
                 size='monetary', 
                 color='Segment', 
                 hover_data=['frequency', 'monetary'],
                 title='Customer Segmentation: RFM vs LTV',
                 labels={'recency': 'Recency (Days since last purchase)', 'LTV': 'Customer Lifetime Value'},
                 size_max=15)

fig.update_layout(xaxis_title='Recency (Days)',
                  yaxis_title='Customer Lifetime Value',
                  legend_title_text='RFM Segment')

fig.show()

In [30]:
customer_info_df = pd.read_csv(os.path.join(current_directory, "../1_Preprocessing Notebooks/Final Sheets/customer_details.csv"))

customer_info_df.drop(customer_info_df[customer_info_df['deceased_indicator']=='Yes'].index, inplace=True)
customer_info_df.drop('deceased_indicator', axis=1, inplace=True)
customer_info_df['job_title'].fillna('N/A', inplace = True)
customer_info_df['job_industry_category'].fillna('N/A', inplace = True)

merged_df = pd.merge(rfm_ltv_df, customer_info_df, on='customer_id', how='inner')
merged_df

,customer_id,recency,frequency,monetary,RFM_rank,Segment,LTV,name,gender,past_3_years_bike_related_purchases,...,job_industry_category,wealth_segment,owns_car,tenure,building_number,street_name,postcode,state,country,property_valuation
0,1,8,11,9084.45,555,Champions,8760.894247,Laraine Medendorp,Female,93,...,Health,Mass Customer,Yes,11,60,Morning Avenue,2016,New South Wales,Australia,10
1,2,129,3,4149.07,112,Hibernating,1273.139288,Eli Bockman,Male,81,...,Financial Services,Mass Customer,Yes,16,6,Meadow Vale Court,2153,New South Wales,Australia,10
2,4,196,2,1047.72,111,Hibernating,218.155397,Talbot,Male,33,...,IT,Mass Customer,No,7,0,Holy Cross Court,4211,Queensland,Australia,9
3,5,17,6,5903.20,433,Loyal Customers,4625.521096,Sheila-kathryn Calton,Female,56,...,N/A,Affluent Customer,Yes,8,17979,Del Mar Point,2448,New South Wales,Australia,4
4,6,65,5,5931.69,223,Hibernating,4420.327890,Curr Duckhouse,Male,35,...,Retail,High Net Worth,Yes,13,9,Oakridge Court,3216,Victoria,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,3496,257,4,4725.38,112,Hibernating,543.742356,Danya Burnyeat,Male,99,...,Manufacturing,Mass Customer,Yes,19,2565,Caliangt Point,2171,New South Wales,Australia,9
3482,3497,53,3,3744.07,311,New Customers,697.525370,Thia O'Day,Female,73,...,Manufacturing,Affluent Customer,Yes,18,96,Delladonna Trail,3976,Victoria,Australia,5
3483,3498,128,6,5177.06,132,At Risk,2879.296384,Lois Abrahim,Female,28,...,Manufacturing,Mass Customer,No,5,3,Nova Point,3012,Victoria,Australia,4
3484,3499,52,7,7673.48,344,Loyal Customers,6327.993096,Shelton Tewkesberrie,Male,29,...,Manufacturing,Mass Customer,Yes,7,310,Stephen Terrace,4073,Queensland,Australia,9


In [31]:
merged_df.columns

Index(['customer_id', 'recency', 'frequency', 'monetary', 'RFM_rank',
       'Segment', 'LTV', 'name', 'gender',
       'past_3_years_bike_related_purchases', 'age', 'job_title',
       'job_industry_category', 'wealth_segment', 'owns_car', 'tenure',
       'building_number', 'street_name', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')

In [32]:
new_column_order = ['customer_id', 'name', 'gender', 'past_3_years_bike_related_purchases', 'age', 'job_title', 'job_industry_category',
                    'wealth_segment', 'owns_car', 'tenure', 'building_number', 'street_name', 'postcode', 'state', 'country', 'property_valuation',
                    'recency', 'frequency', 'monetary', 'RFM_rank', 'Segment', 'LTV']
merged_df = merged_df[new_column_order]
merged_df

,customer_id,name,gender,past_3_years_bike_related_purchases,age,job_title,job_industry_category,wealth_segment,owns_car,tenure,...,postcode,state,country,property_valuation,recency,frequency,monetary,RFM_rank,Segment,LTV
0,1,Laraine Medendorp,Female,93,64,Executive Secretary,Health,Mass Customer,Yes,11,...,2016,New South Wales,Australia,10,8,11,9084.45,555,Champions,8760.894247
1,2,Eli Bockman,Male,81,37,Administrative Officer,Financial Services,Mass Customer,Yes,16,...,2153,New South Wales,Australia,10,129,3,4149.07,112,Hibernating,1273.139288
2,4,Talbot,Male,33,56,N/A,IT,Mass Customer,No,7,...,4211,Queensland,Australia,9,196,2,1047.72,111,Hibernating,218.155397
3,5,Sheila-kathryn Calton,Female,56,40,Senior Editor,N/A,Affluent Customer,Yes,8,...,2448,New South Wales,Australia,4,17,6,5903.20,433,Loyal Customers,4625.521096
4,6,Curr Duckhouse,Male,35,51,N/A,Retail,High Net Worth,Yes,13,...,3216,Victoria,Australia,9,65,5,5931.69,223,Hibernating,4420.327890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,3496,Danya Burnyeat,Male,99,31,Editor,Manufacturing,Mass Customer,Yes,19,...,2171,New South Wales,Australia,9,257,4,4725.38,112,Hibernating,543.742356
3482,3497,Thia O'Day,Female,73,31,Administrative Assistant IV,Manufacturing,Affluent Customer,Yes,18,...,3976,Victoria,Australia,5,53,3,3744.07,311,New Customers,697.525370
3483,3498,Lois Abrahim,Female,28,22,N/A,Manufacturing,Mass Customer,No,5,...,3012,Victoria,Australia,4,128,6,5177.06,132,At Risk,2879.296384
3484,3499,Shelton Tewkesberrie,Male,29,38,N/A,Manufacturing,Mass Customer,Yes,7,...,4073,Queensland,Australia,9,52,7,7673.48,344,Loyal Customers,6327.993096


In [33]:
fig = px.scatter(merged_df, x='monetary', y='LTV', color='Segment', 
                 title='LTV vs Monetary Value by RFM Segment',
                 hover_data=['name', 'age', 'job_title'])
fig.show()

In [34]:
fig = px.pie(merged_df, names='wealth_segment', title='Customer Distribution by Wealth Segment', hole=0.4)
fig.show()


In [35]:
fig = px.box(merged_df, x='wealth_segment', y='LTV', color='wealth_segment', 
             title='LTV Distribution Across Wealth Segments')
fig.show()


In [36]:
# Group by RFM Segment and Gender, then calculate average LTV
avg_ltv_gender_segment = merged_df.groupby(['Segment', 'gender']).agg({'LTV': 'mean'}).reset_index()

fig = px.bar(avg_ltv_gender_segment, x='Segment', y='LTV', color='gender', 
             barmode='group', title='Average LTV by RFM Segment and Gender',
             labels={'LTV': 'Average LTV', 'gender': 'Gender'})
fig.show()


In [37]:
import plotly.figure_factory as ff

# Select relevant columns for correlation
correlation_df = merged_df[['recency', 'frequency', 'monetary', 'LTV']].corr()

fig = ff.create_annotated_heatmap(z=correlation_df.values, 
                                  x=list(correlation_df.columns), 
                                  y=list(correlation_df.index),
                                  colorscale='Viridis')
fig.update_layout(title='Correlation Heatmap Between RFM Metrics and LTV')
fig.show()


In [38]:
fig = px.treemap(merged_df, 
                 path=['wealth_segment', 'job_industry_category', 'Segment'], 
                 values='customer_id', 
                 title='Customer Distribution by Wealth Segment, Job Industry, and RFM Segment',
                 hover_data=['LTV', 'age'])
fig.show()


In [39]:
fig = px.box(merged_df, x='Segment', y='age', color='wealth_segment', 
             title='Age Distribution by RFM Segment and Wealth Segment',
             labels={'age': 'Customer Age'})
fig.show()

In [40]:
merged_df.to_csv('RFM Segment & LTV.csv', index=False)